In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn import set_config
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_object_dtype
from sklearn.preprocessing import OneHotEncoder, StandardScaler
set_config(display="diagram")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [47]:
train_filename='../input/spaceship-titanic/train.csv'
test_filename='../input/spaceship-titanic/test.csv'

Let's convert the data in Pandas dataframe and start working

In [48]:
train_data= pd.read_csv(train_filename)
test_data= pd.read_csv(test_filename)

Let's see the data

In [49]:
train_data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Let's 
PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.

HomePlanet - The planet the passenger departed from, typically their planet of permanent residence. This is an catagorical variable.

CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins. 

Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.

Destination - The planet the passenger will be debarking to.

Age - The age of the passenger.

VIP - Whether the passenger has paid for special VIP service during the voyage.

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

Name - The first and last names of the passenger.

Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

Data Analysis

In [50]:
train_data.shape

(8693, 14)

So this dataset has 8693 rows and 14 columns.

Next step we will check for missing values in the dataset if present and datatypes of each column

In [51]:
train_data.info()# to check datatypes 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [52]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


Columns have 3 different kind of datatypes: object, float64 and bool. We can see that other than 'PassengerId' and 'Transported' columns other columns have missing values so we need to take care of that. Next step, we will count number of missing values in each column.

In [53]:
train_data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

At the time of data cleaning we will impute missing values

In [54]:
train_data.isnull().sum()/train_data.shape[0]

PassengerId     0.000000
HomePlanet      0.023122
CryoSleep       0.024963
Cabin           0.022892
Destination     0.020936
Age             0.020591
VIP             0.023352
RoomService     0.020821
FoodCourt       0.021051
ShoppingMall    0.023927
Spa             0.021051
VRDeck          0.021627
Name            0.023007
Transported     0.000000
dtype: float64

Usually When a column has more than 80% to 95% missing value, we drop it but here missing values are in 2-3%, so it is not a good idea to drop any column here.

Next we will check some statistical informations about each column.

In [55]:
train_data.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [56]:
train_data.describe(include=['O'])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name
count,8693,8492,8476,8494,8511,8490,8493
unique,8693,3,2,6560,3,2,8473
top,0001_01,Earth,False,G/734/S,TRAPPIST-1e,False,Gollux Reedall
freq,1,4602,5439,8,5915,8291,2


'PassengerId'and 'Name'are unique for everyone. So we will drop them as they are not important for further work. 

We will break down feature and target variable first,then work more on target variable to get the finer information about features are influencing target variabe.

In [57]:
train_data=train_data.drop(['PassengerId','Name'],axis=1)


In [58]:
train_data[['Deck', 'Num', 'Side']] = train_data['Cabin'].str.split('/', expand=True)
train_data=train_data.drop('Cabin',axis=1)

test_data[['Deck', 'Num', 'Side']] = test_data['Cabin'].str.split('/', expand=True)
test_data=test_data.drop('Cabin',axis=1)


In [59]:
train_data.describe(include=['O'])

,HomePlanet,CryoSleep,Destination,VIP,Deck,Num,Side
count,8492,8476,8511,8490,8494,8494,8494
unique,3,2,3,2,8,1817,2
top,Earth,False,TRAPPIST-1e,False,F,82,S
freq,4602,5439,5915,8291,2794,28,4288


In [60]:
train_data=train_data.drop('Num',axis=1)
test_data=test_data.drop('Num',axis=1)

In [61]:
train_data['total_spent']= train_data['RoomService']+ train_data['FoodCourt']+ train_data['ShoppingMall']+ train_data['Spa']+ train_data['VRDeck']
test_data['total_spent']= test_data['RoomService']+ test_data['FoodCourt']+ test_data['ShoppingMall']+ test_data['Spa']+ test_data['VRDeck']


In [62]:
cols=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
train_data=train_data.drop(cols,axis=1)
test_data=test_data.drop(cols,axis=1)

**Conclusion:** After the data analysis, we have 8 features(Homeplanet, cryosleep,Deck,side,Destination,age,VIP,total_spent) which have relation with transported member count.

Data Preprocessing

In [63]:
#feature and target variable separation

X=train_data.loc[:,['HomePlanet','CryoSleep', 'Destination','Age','VIP' ,'Deck' ,'Side','total_spent']]
y=train_data.loc[:,'Transported']
print(X)

     HomePlanet CryoSleep    Destination   Age    VIP Deck Side  total_spent
0        Europa     False    TRAPPIST-1e  39.0  False    B    P          0.0
1         Earth     False    TRAPPIST-1e  24.0  False    F    S        736.0
2        Europa     False    TRAPPIST-1e  58.0   True    A    S      10383.0
3        Europa     False    TRAPPIST-1e  33.0  False    A    S       5176.0
4         Earth     False    TRAPPIST-1e  16.0  False    F    S       1091.0
...         ...       ...            ...   ...    ...  ...  ...          ...
8688     Europa     False    55 Cancri e  41.0   True    A    P       8536.0
8689      Earth      True  PSO J318.5-22  18.0  False    G    S          0.0
8690      Earth     False    TRAPPIST-1e  26.0  False    G    S       1873.0
8691     Europa     False    55 Cancri e  32.0  False    E    S       4637.0
8692     Europa     False    TRAPPIST-1e  44.0  False    E    S       4826.0

[8693 rows x 8 columns]


In [64]:
#categorical and numerical column separation
num_cols=list(X.select_dtypes(exclude='object').columns)
cat_cols=list(X.select_dtypes(include='object').columns)
print(num_cols)

['Age', 'total_spent']


In [65]:
#pipeline preparation
numerical_pipe =Pipeline(steps=[
    ('norm', StandardScaler()),
    ('imputer', SimpleImputer(strategy='mean'))
    ])
categorical_pipe =  Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('norm', StandardScaler(with_mean = False))
])
preprocessors = ColumnTransformer(transformers=[
    ('num', numerical_pipe, num_cols),
    ('cat', categorical_pipe, cat_cols)
])

pipe1 = Pipeline([
    ('preprocessors', preprocessors),
    ('model1',
     KNeighborsClassifier(n_neighbors = 15, metric = 'minkowski', p = 2))])

pipe2 = Pipeline([
    ('preprocessors', preprocessors),
    ('model2',DecisionTreeClassifier(criterion = 'entropy', random_state = 0))])

pipe3 = Pipeline([
    ('preprocessors', preprocessors),
    ('model3',RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0))])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state = 0) 

# fitting the model and making prediction
pipe1.fit(X_train,y_train)                                                    
y_pred1=pipe1.predict(X_test)

pipe2.fit(X_train,y_train)                                                    
y_pred2=pipe2.predict(X_test)

pipe3.fit(X_train,y_train)                                                    
y_pred3=pipe3.predict(X_test)
print('done')


done


Confusion matrix and accuracy score

In [66]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm1 = confusion_matrix(y_test, y_pred1)
cm2 = confusion_matrix(y_test, y_pred2)
cm3 = confusion_matrix(y_test, y_pred3)
print("confusion matrix for K-NN classifier")
print(cm1)
print("confusion matrix for decision tree classifier")
print(cm2)
print("confusion matrix for random forest classifier")
print(cm3)
print("accuracy for K-NN classifier")
print(accuracy_score(y_test, y_pred1))
print(" accuracy for decision tree classifier")
print(accuracy_score(y_test, y_pred2))
print("accuracy for random forest  classifier")
print(accuracy_score(y_test, y_pred3))

confusion matrix for K-NN classifier
[[890 187]
 [378 719]]
confusion matrix for decision tree classifier
[[715 362]
 [341 756]]
confusion matrix for random forest classifier
[[796 281]
 [359 738]]
accuracy for K-NN classifier
0.7401103955841767
 accuracy for decision tree classifier
0.6766329346826127
accuracy for random forest  classifier
0.7056117755289788


As the K-NN classifier performs best, so we will use that for future prediction

In [67]:
test_pred = pipe1.predict(test_data.drop(['PassengerId', 'Name'], axis=1))
pd.DataFrame({'PassengerId' : test_data.PassengerId,
              'Transported' : test_pred}
            ).to_csv('save.csv', index=False)